# FUNZIONE DI FORECASTING

                make_forecast(

                df,
                forecast_from, 
                forecast_to,
                freq='D',

                **kwargs:
                # preprocessing
                preprocess_from, 
                preprocess_to, 
                ds, 
                target = kwargs.get('target') 

                # Prophet args
                growth = kwargs.get('growth', 'linear')
                cap  = kwargs.get('cap')
                floor  = kwargs.get('floor')
                lockdowns  = kwargs.get('lockdowns') 

                # additional regressors what if
                regressors = kwargs.get('regressors') 
                replace_regressors_values = kwargs.get('replace_regressors_values') 

                # additional regressors to be forecasted
                to_forecast = kwargs.get('to_forecast') 
                mode = kwargs.get('mode')
                
                # layout
                title  = kwargs.get('title')
                )

## 1. INPUT MINIMI OBBLIGATORI

In questa sezione si descriveranno gli input minimi necessari alla funzione per svolgere il forecast.

### 1.1 DATA FRAME
Il data frame è il nome generale con la quale viene indicata la base dati sotto forma di tabella relazionale, in python questa viene ottenuta mediante la libreria Pandas.

#### 1.1.2 ANALISI UNIVARIATA
Il dataframe di input per il forecast univariato ha bisogno di almeno 2 colonne: 
- **tempo:** 
la granularità del dataset dipende da questa colonna, la funzione accetta 2 tipi di granularità: giornaliera e mensile. È fondamentale che in questa colonna non vi siano buchi temporali o giorni mancanti. Qualora vi siano vanno imputati i giorni mancanti riportando valore 0 nelle variabili corrispondenti.
Idealmente il nome della colonna dovrebbe essere 'ds', tuttavia nella funzione è predisposto un argomento per rinominarla conformemente a Prophet. 
Il dtype della colonna dev'essere del tipo datetime, qualora la colonna non fosse di questo tipo di dato rimando alla documentazione di pandas al seguente link: 
https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html
- **target:** il target è la colonna che vogliamo prevedere, idealmente il nome della colonna dovrebbe essere 'y', tuttavia, analogamente alla colonna di tempo, nella funzione è predisposto un argomento per rinominarla conformemente a Prophet. Il dtype della colonna deve essere necessariamente int o float, qualora non lo fosse: df['col'].astype(float). Il forecast deve avvenire per brand e per region/country.

#### 1.1.2 ANALISI MULTIVARIATA

Per l'analisi multivariata, oltre alle due colonne sopra menzionate per il forecast univariato, sarà necessario aggiungere una colonna per ogni regressore che si voglia utilizzare per questo tipo di forecast. È di primaria importanza che i regressori riportino solo ed esclusivamente i valori relativi alla stagione dell'analisi in corso. Qualora vi fossero valori mancanti o non inerenti sarà necessario azzerarli mantenendo tuttavia la linearità storica invariata (no buchi temporali).

L'analisi multivariata può essere di tre tipi:
- **multi-variata what if:** i valori futuri dei regressori possono essere modificati dall'analyst a seconda dei valori attesi. Questa sostituzione può avvenire sia imputando i valori relativi alla stagione precedente lasciandoli invariati o agendo su di essi con variazioni percentuali applicate all'intero periodo di forecast. Al contempo è stato predisposto un argomento della funzione per permettere la variazione del singolo valore di un quadrante (riga, colonna), approfondimento nel paragrafo KWARGS.
- **multi-variata forecast**: qualora i regressori non siano dipendenti da fattori intrinseci e pertanto dipendenti dalle volontà del management sarà necessario prevedere gli stessi per permettere il forecast multivariato. Anche per questo tipo di analisi è stato predisposto un argomento della funzione per permettere ciò, approfondimento nel paragrafo KWARGS.
- **ibrido**: what if + forecast

#### 1.1.3 PREPROCESSING MULTIVARIATA

Per ottenere il data frame corretto da imputare per il forecast multivariato sarà necessario seguire alcuni step di preprocessing.
Anzitutto va specificato che qualora si volesse effettuare il forecast multivariato con tutti i KPI che al momento sono stati individuati come statisticamente rilevanti e privi di multicollinearità (Spending in performance marketing e avg_discount) può solo avere granularità mensile. Ciò è dovuto al fatto che i dati di Spending vengono raccolti mensilmente. Qualora si volesse quindi effettuare un forecast multivariato con solo avg_discount come unico regressore esogeno sarà possibile farlo a granularità giornaliera.

- **avg_discount**: estratto da wbi fatturato, ha granularità giornaliera e colonna di stagione. Sarà sufficente creare 2 data frame, uno per stagione, fare il merge con i dati di fatturato ed imputare con 0 eventuali NaN.
- **Spending**: fonte file excel dei relativi team di performance marketing per brand. I dati di Spending ad oggi non vengono raccolti segnalando la stagione a cui si riferiscono pertanto si rende necessario uno step di preprocessing a tal proposito. Il periodo che va da Marzo ad Agosto è relativo a campagne che hanno come soggetto la collezione Spring Summer e pertanto lo Spending sarà attribuito ad essa, viceversa i mesi da Settembre a Febbraio saranno attribuiti a Fall Winter. Una volta creati i due data frame per stagione sulla base di questa distinzione ed effettuato il merge con i dati di fatturato ed eventualmente quelli di avg discount, rimarrà soltanto da raggruppare i dati a granularità giornaliera in modo uniformare il dataset alla granularità mensile.

**Best practice**: benchè da convenzione venga riportata la data del primo giorno del mese di riferimento relativo al dato di Spending di performance marketing, per come funziona Prophet è opportuno che venga riportata la data di fine mese. Ciò può essere fatto indicando alla funzione di groupby di aggregare le date per anno e mese riportando la data massima indicata nella colonna di data stamp:  
    
    es: df.groupby(['year','month'])['ds','avg_discount','Spending','y'].agg({'ds':'max','avg_discount':'mean','Spending':'sum','y':'sum'})

**Nice to have**: la segnalazione della stagione di riferimento dei  dati di performance marketing da parte del team in una colonna apposita non solo faciliterebbe il lavoro di preprocessing ma eviterebbe eventuali distorsioni derivanti da ricostruzioni fatte a posteriori magari a distanza di anni e/o da assunzioni che potrebbero non valere nel caso specifico i brand o di region. _(es. emisfero australe ss/fw invertite? Brand che iniziano/concludono campagne con periodi differenti?)_

#### 1.2 FORECAST FROM

Forecast from è il secondo _positional argument_ obbligatorio da inserire nella funzione. Esso va indicato sotto forma di stringa e deve essere coerente con il formato data utilizzato nella colonna di datastamp del data frame imputato. Come suggerisce il nome indica alla funzione la data dalla quale iniziare a fare il forecast, a seconda di quando questa ricada relativamente al data frame di riferimento la funzione darà in output casistiche differenti:
- **forecast_from < ultima data presente el data frame**: casistica di testing. Questo è il caso in cui si voglia testare l'accuratezza del modello. Ciò avviene confrontando i dati di output del forecast con quelli effettivamente verificatisi. Per quantificare il risultato vengono restituiti una serie di KPI che verranno descritti nella sezione 1.3.
- **forecast_from = al giorno/mese successivo l'ultimo riportato nel data frame**: casistica tipo di forecast operativo, gli output saranno relativi l'intero periodo di riferimento compreso fra forecast_from e forecast_to. A seconda di quando ricada quest ultimo il periodo di paragone del forecast sarà la stagione conclusa più recente.
- **forecast_from > del giorno/mese successivo l'ultimo riportato nel data frame**: casistica tipo di forecast operativo, gli output saranno unicamente relativi l'intero periodo di riferimento compreso fra forecast_from e forecast_to. A seconda di quando ricada quest ultimo il periodo di paragone del forecast sarà la stagione conclusa più recente. Dettaglio tecnico: dettato dal funzionamento di Prophet, il forecast reale avviene comunque a partire dal primo giorno non presente nell data frame di input, tuttavia i KPI e la data visualization in output saranno relativi al solo periodo di riferimento.

#### 1.3 FORECAST TO

L'ultimo positional argument da inserire obbligatoriamente è forecast_to. Come suggerisce il nome è la data fino alla quale verrà effettuato il forecast, essa sarà inclusa nell'intervallo. Analogamente a forecast_from anche in questo caso il formato deve essere coerente con la colonna di data stamp. A seconda di quando questa cada rispettivamente al data frame imputato si apriranno scenari differenti:
- **forecast_to entro l'anno successivo del dataset in input**: qualora il forecast non superi la durata di un anno dall'ultima data riportata nel data frame di input, il forecast verrà paragonato con la stagione di riferimento precedente.
- **forecast_to oltre l'anno successivo del dataset in input**: qualora il forecast superi la durata di un anno dall'ultima data riportata nel data frame di input, il forecast verrà paragonato con la stagione di riferimento di due anni prima. Ciò avviene per assicurare maggiore uniformità nel confronto in modo da paragonare solo mensilità relative allo stesso macro periodo o stagione.

## 2. KWARGS

I KWARGS sono argomenti opzionali della funzione. A differenza dei positional argument non devono essere inseriti in un ordine preciso, tuttavia necessitano di essere preceduti dalla loro keyword identificativa.

#### 2.1 PREPROCESSING:

I seguenti KWARGS avranno effetto su data frame in input:
- **preprocess_from**: data dalla quale si vuole che il data frame in input inizi. _N.B. il formato data deve essere uguale a quello utilizzato nel data frame, la data indicata è inclusa nel data frame_
- **preprocess_to**: data dalla quale si vuole che il data frame in input finisca. _N.B. il formato data deve essere uguale a quello utilizzato nel data frame, la data indicata è inclusa nel data frame_
- **ds**: per indicare sotto forma di stringa il nome della colonna contenente il data stamp. Questa colonna verrà rinominata in 'ds' come richiesto da Prophet.
- **target**: per indicare sotto forma di stringa il nome della colonna obiettivo del forecast. Questa colonna verrà rinominata in 'y' come richiesto da Prophet.

#### 2.2 PROPHET

I seguenti KWARGS avranno effetto sui parametri di Prophet:
- **freq**: _def. 'M', dtype: str_ indica la frequenza/granularità del forecast e del data frame input. Può essere settato su 'M' per forecast mensile o su 'D' per forecast giornaliero
- **growth**: _def. 'linear', dtype: str_ indica il tipo di crescita del modello, 'linear' indica crescita lineare, 'logistic' logistica, per quest'ultima sarà necessario indicare anche _cap_ e _floor_ .
- **cap**: _dtype: int, float_ upper bound del forecast
- **floor**: _dtype: int, float_ lower bound del forecast
- **lockdowns**: _dtype: str_ nome del data frame contenente il/i periodo/i da segnalare come one-off-holiday: https://facebook.github.io/prophet/docs/handling_shocks.html#treating-covid-19-lockdowns-as-a-one-off-holidays

#### 2.3 REGRESSORI AGGIUNTIVI PER ANALISI WHAT IF:
- **regressors**: _dtype: dictionary es. {'reg1':1.0, 'reg2':1.15}_ dev'essere un dizionario contenente come chiavi i nomi del/i regressore/i esogeno/i da aggiungere al modello. Le chiavi del dizionario devono essere colonne del data frame di input. I valori corrispettivi sono i fattori da moltiplicare a ciascun valore relativo al periodo precedente del regressore di riferimento. Qualora forecast_to sia successivo ad un anno dall'ultima data riportata nel data frame i valori del regressore saranno quelli dello stesso intervallo temporale del forecast ma di due anni prima, altrimenti saranno riferiti all'anno precedente.
- **replace_regressors_values**: _dtype: nested dictionary es. {'reg1':{'data1':val1,'data2':val2}, 'reg1':{'data1':val1,'data2':val2}}_ permette di modificare i singoli valori relativi a quel regressore per quella data specifica. La chiave primaria del dizionario dev'essere una stringa indicante il nome della colonna per la quale si vuole la sostituzione del valore. La data deve essere presente nel data frame e deve essere scritta nello stesso formato del data frame, altrimenti la sostituzione non avverrà. 

#### 2.4 REGRESSORI AGGIUNTIVI DA FORECASTARE A LORO VOLTA

Qualora vi fossero regressori esogeni i quali valori futuri non possano essere stimati o imputati dall'utente sarà necessario che questi vengano a loro volta forecastati. Al momento il forecast avviene in maniera autoregressiva univariata per ciascuno di essi. Non è da escludere in futuro la possibilità di implementare la possibilità di forecast multivariato anche su di essi. La presenza di questo keyword argument aggiungerà agli output il line plot del regressore forecastato paragonato al dato reale dello stesso periodo di riferimento (qualora si tratti di caso test) o paragonato al dato di uno o due anni precedente a seconda che forecast_to sia di uno o più anni successivo all'ultima data riportata nel data frame in input.

- **to_forecast**: _dtype: list_ contenente stringhe riportanti i nomi delle colonne relativi ai regressori  da forecastare che si vogliano aggiungere all'analisi multivariata.
- **mode**: _dtype: string_ mode può essere o 'additive' (default) o 'multiplicative', il kwargs avrà affetto su tutti i regressori.

#### 2.5 LAYOUT

Argomenti per modificare il layout dei grafici in output

- **title**: serve per modificare il titolo del barplot di comparazione fra forecast e dato reale

## 3. OUTPUT

Gli output della funzione variano a seconda del forecast specifico, tuttavia vi sono comunque dati e KPI che saranno sempre forniti in output.

### 3.1 OUTPUT IN COMUNE

- **tot forecast**: valore aggregato del target nel periodo di riferimento.
- **valore di paragone**: valore per avere un confronto fra un dato effettivamente osservato ed il  valore di forecast.
- **delta valore reale e di forecast**: espresso sia in termini reali che percentuali.
- **tabella pivot**: riportante il dato aggregato per mese in riga e per anno in colonna.
- **tabella di forecast**: riportante dato di forecast con lower e upper bound espresso sia in termini monetari che percentuali.
- **pivot visualizzata con plotly**
- **lineplot**: riportante il dato forecastato (linea sottile blu), fascia azzurro chiara di lower e upper bound, dato reale riportato con dot neri.
- **bar plot**: riportante il dato di forecast confrontato con dato reale. 
- **components plot**: riportante trent annuali, mensili, settimanali e nel caso in cui vi siano apporto dei regressori.

### 3.2 OUTPUT WHAT IF

- **regressori aggiuntivi**: elenco regressori esogeni utilizzati nel forecast multivariato con relativi fattori di moltiplicazione.
- **data frame regressori esogeni**: data frame riportante tutti i valori utilizzati per il forecast multivariato.

### 3.3 OUTPUT MULTIVARIATO FORECAST

- **lineplot regressore**: line plot per la visualizzazione del forecast univariato del regressore da utilizzarsi per il forecast multivariato.

### 3.4 OUTPUT TEST

- **The Root Mean Squared Error**: valore medio dello scostamento assoluto del forecast dal valore reale per il periodo oggetto, ottenuto elevando alla seconda l'errore. Questa misura penalizza scostamenti più elevati in valore assoluto.
- **The Mean Absolute Error**: valore medio dello scostamento assoluto del forecast dal valore reale per il periodo oggetto, ottenuto facendo la media del valore assoluto dell'errore. Questa misura non penalizza la magnitudine dell'errore. _N.B. minore sarà la differenza fra RMSE e MAE più vorrà dire che l'errore sarà di entità minore rispetto al valore totale del forecast sull'intero periodo_
- **The OVERALL MAPE**: media degli scostamenti assoluti del forecast dai valori reali di ogni punto di forecast. 
- **The OVERALL WAPE**: somma dei loss del singolo periodo sul valore reale dell'intero periodo.
- **The OVERALL WAPE MEAN**: media dei loss del singolo periodo sul valore reale dell'intero periodo.
- **regressori aggiuntivi**: elenco regressori esogeni utilizzati nel forecast multivariato con relativi fattori di moltiplicazione.
- **data frame regressori esogeni**: data frame riportante tutti i valori utilizzati per il forecast multivariato.
- **data frame valori reali**: riferito all'intero periodo oggetto di forecast.
- **data frame valori forecast**: riferito all'intero periodo oggetto di forecast con upper e lower bound.
- **final data frame**: di confronto riferito all'intero periodo oggetto di forecast con upper, lower bound e wape.
- **final data frame plotly**: di confronto riferito all'intero periodo oggetto di forecast con upper, lower bound e wape.
- **lineplot**: riportante dato reale in rosa.
- **barplot**: riportante dato reale in arancione.
- **components plot**: riportante trent annuali, mensili, settimanali e nel caso in cui vi siano apporto dei regressori.


# IMPORTS

In [1]:
# IMPORTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from statsmodels.tools.eval_measures import rmse
from prophet.plot import plot_plotly, plot_components_plotly, add_changepoints_to_plot
import plotly.graph_objects as go
import plotly.express as px
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error
from scipy import stats
from prophet.utilities import regressor_coefficients
from dateutil.relativedelta import relativedelta

# DEF FUNC

In [2]:
def make_forecast(df,
                  forecast_from, # INCLUDED, if forecast from > than the last day of the given df, data viz starts from that day
                  forecast_to,
                  freq='D', # INCLUDED
                  **kwargs):

    # IN COMUNE MONO/MULTI

    # KWARGS

    # Date
    preprocess_from = kwargs.get('preprocess_from')
    preprocess_to = kwargs.get('preprocess_to')

    # targets
    ds = kwargs.get('ds') # colonna del df da rinominare in ds nel caso in cui non lo fosse
    target = kwargs.get('target') # colonna del df da rinominare in y nel caso in cui non lo fosse
    # Prophet args
    #freq= kwargs.get('freq', 'M')
    growth = kwargs.get('growth', 'linear')
    cap  = kwargs.get('cap')
    floor  = kwargs.get('floor')
    lockdowns  = kwargs.get('lockdowns') # one off holidays
    # additional regressors what if
    regressors = kwargs.get('regressors') # dictionary pair of regressors and floats to multiply all the values of the regressor
    replace_regressors_values = kwargs.get('replace_regressors_values') # must be a nested disctionary 
    # additionnal regressors to e forcasted
    to_forecast = kwargs.get('to_forecast') # list like
    mode = kwargs.get('mode')
    # layout
    title  = kwargs.get('title')

    # da strinnga a datetime
    preprocess_from = pd.to_datetime(preprocess_from)
    preprocess_to = pd.to_datetime(preprocess_to)
    #start_date = pd.to_datetime(start_date)
    #end_date = pd.to_datetime(end_date)
    forecast_from = pd.to_datetime(forecast_from)
    forecast_to = pd.to_datetime(forecast_to)

    if preprocess_to==None:
        preprocess_to = df['ds'].iloc[-1]
    if preprocess_to!=None:
        df = df.loc[(df['ds'] <= preprocess_to)]
    if preprocess_from!=None:
        df = df.loc[(df['ds'] >= preprocess_from)]
        
    if forecast_from > (df['ds'].iloc[-1] + relativedelta(days=1)):
        forecast_from_oya = pd.to_datetime(pd.to_datetime(df['ds'].iloc[-1])-relativedelta(years=1))
        forecast_from_tya = pd.to_datetime(pd.to_datetime(df['ds'].iloc[-1])-relativedelta(years=2))
        forecast_from_oya0 = pd.to_datetime(forecast_from)-relativedelta(years=1)
        forecast_from_tya0 = pd.to_datetime(forecast_from)-relativedelta(years=2) 
    else:
        forecast_from_oya = pd.to_datetime(forecast_from)-relativedelta(years=1)
        forecast_from_tya = pd.to_datetime(forecast_from)-relativedelta(years=2)
        
    forecast_to_oya = pd.to_datetime(pd.to_datetime(forecast_to)-relativedelta(years=1))
    forecast_to_tya = pd.to_datetime(pd.to_datetime(forecast_to)-relativedelta(years=2))

    # DEF FUNCTIONS
    
    # mape
    def mape(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return str(np.mean(np.abs((y_true - y_pred))/y_true) * 100) + ' % of error'

    # wape
    def wape_mean(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return str((np.mean(np.abs(y_true-y_pred))/np.abs(np.sum(y_true)))* 100) + ' % of error'

    def wape(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return str((np.sum(np.abs(y_true-y_pred))/np.abs(np.sum(y_true)))* 100) + ' % of error'
    
    # replace regressors values
    def rrv(df, dict):
        for i,j in dict.items():
            for r in j:
                k = pd.to_datetime(r)
                df[i].loc[df['ds']==k] = j[r]
        return df

    # PREPROCESSING 
    df = df.rename({ds:'ds', target:'y'}, axis=1)

    if forecast_from > (df['ds'].iloc[-1] + relativedelta(days=1)):
        # dates to create future forecast
        idx0 = pd.date_range(df['ds'].iloc[-1] + relativedelta(days=1), forecast_to).to_frame()
        future_days0 = len(idx0)
        idx0['month'] = idx0.index.month
        idx0['year'] = idx0.index.year
        idx0=idx0.groupby(['year','month'])[idx0.columns].max()
        n_months0 = len((idx0['month'])) # .unique()
        # dates for kpis and data viz
        idx = pd.date_range(forecast_from, forecast_to).to_frame()
        future_days = len(idx)
        idx['month'] = idx.index.month
        idx['year'] = idx.index.year
        idx=idx.groupby(['year','month'])[idx.columns].max()
        n_months = len((idx['month']))
    else:
        idx = pd.date_range(forecast_from, forecast_to).to_frame()
        future_days = len(idx)        
        idx['month'] = idx.index.month
        idx['year'] = idx.index.year
        idx=idx.groupby(['year','month'])[idx.columns].max()
        n_months = len((idx['month']))
        
    # aggiungo colonnna per crescita logistica
    df['floor']=floor
    df['cap']=cap

    # setting the model
    m = Prophet(
        growth=growth,
        holidays=lockdowns,
        #interval_width=0.95
    )
    # --------------------------------------------------                       ∆                       --------------------------------------------------
    # --------------------------------------------------                      ∆∆∆                      --------------------------------------------------
    # --------------------------------------------------                     ∆∆∆∆∆                     --------------------------------------------------
    # --------------------------------------------------                    ∆∆∆∆∆∆∆                    --------------------------------------------------
    # --------------------------------------------------                   ∆∆∆∆∆∆∆∆∆                   --------------------------------------------------
    # --------------------------------------------------                  ∆∆∆∆∆∆∆∆∆∆∆                  --------------------------------------------------
    # --------------------------------------------------                 ∆∆∆∆∆∆∆∆∆∆∆∆∆                 --------------------------------------------------
    # --------------------------------------------------                ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆                --------------------------------------------------
    # --------------------------------------------------               ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆               --------------------------------------------------
    # --------------------------------------------------              ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆              --------------------------------------------------
    # --------------------------------------------------             ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆             --------------------------------------------------
    # --------------------------------------------------            ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆            --------------------------------------------------
    # --------------------------------------------------           ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆           --------------------------------------------------
    # --------------------------------------------------          ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆          --------------------------------------------------
    # --------------------------------------------------         ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆         --------------------------------------------------
    # --------------------------------------------------        ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆        --------------------------------------------------
    # --------------------------------------------------       ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆       --------------------------------------------------
    # --------------------------------------------------      ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆      --------------------------------------------------
    # --------------------------------------------------     ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆     --------------------------------------------------
    # --------------------------------------------------    ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆    --------------------------------------------------
    # --------------------------------------------------   ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆   --------------------------------------------------
    # --------------------------------------------------  ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆  --------------------------------------------------    
    # -------------------------------------------------- ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆ FORECAST CASE ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆ --------------------------------------------------
    
    # forecast_to oltre storico dati stagione precente (no paragone) e forecst_from da oltre un mese da ultima data nel df
    if (forecast_to - relativedelta(years=1)) > df['ds'].iloc[-1] and forecast_from > df['ds'].iloc[-1] + relativedelta(days=1):
        # MULTIVARIATE
        # what if
        if regressors != None:
            for reg in regressors:
                m.add_regressor(reg,
                                mode=mode
                                )

        # forecasted 
        if to_forecast != None:
            def predictRegressor(df, x):
                df = df[["ds", x]]
                new_names = {x: "y"}
                df.rename(columns=new_names, inplace=True)

                m = Prophet()
                m.fit(df)

                #Create dates df for the future
                if freq=='D':
                    future = m.make_future_dataframe(periods = future_days0, freq=freq)
                elif freq=='M':
                    future = m.make_future_dataframe(periods = n_months0, freq=freq)                

                #Model predictions
                forecast = m.predict(future)
                
                # plot
                print('The forecasted regressor:')
                fig = plot_plotly(m, forecast)
                fig.show();
                print(('-----------------------------------------------------------------------------------------'))
                forecast.rename({'yhat':x}, axis=1, inplace=True)
                return forecast 

            for reg2 in to_forecast:
                m.add_regressor(reg2)

        # FITTING
        m.fit(df)

        # creating future df
        if freq=='D':
            future = m.make_future_dataframe(periods = future_days0, freq=freq)
        elif freq=='M':
            future = m.make_future_dataframe(periods = n_months0, freq=freq)

        future['floor']=floor
        future['cap']=cap

        # add additional regressors column
        if regressors != None:
            for reg, value in regressors.items():
                future = future.merge(df[[reg, 'ds']], on='ds',  how='left')
                if freq=='M':
                    future[reg].iloc[-n_months0:] = df[reg].loc[(df['ds']>forecast_from_tya)&(df['ds']<=forecast_to_tya)]*value
                elif freq=='D':
                    future[reg].iloc[-future_days0:] = df[reg].loc[(df['ds']>forecast_from_tya)&(df['ds']<=forecast_to_tya)]*value

        if replace_regressors_values != None:
            rrv(future, replace_regressors_values)

        if to_forecast != None:
            for reg3 in to_forecast:
                forecasted = predictRegressor(df, reg3)
                future[reg3] = forecasted[reg3]

        # predicting
        forecast = m.predict(future)

        # creating columns for groupby
        df['month'] = df['ds'].dt.month
        df['year'] = df['ds'].dt.year
        forecast['month'] = forecast['ds'].dt.month
        forecast['year'] = forecast['ds'].dt.year

        # creating df for results
        grouped_forecast = forecast\
            .loc[(forecast['ds']>=df['ds'].iloc[-1] + relativedelta(days=1)) & (forecast['ds']<=forecast_to)]\
            .groupby(['year', 'month'])['yhat', 'yhat_lower', 'yhat_upper', 'ds']\
            .agg({
                'yhat':'sum',
                'yhat_lower':'sum',
                'yhat_upper':'sum',
                'ds':'max'
                })\
                .reset_index()\
                .rename({'yhat':'y'}, axis=1)

        grouped_df = df\
            .groupby(['year', 'month'])['ds','y']\
            .agg({'ds':'max','y':'sum'}).reset_index().rename({'y':'real'}, axis=1)                

        grouped_forecast['lower_bound_%'] = ((grouped_forecast['yhat_lower']-grouped_forecast['y'])/grouped_forecast['y'])*100
        grouped_forecast['upper_bound_%'] = ((grouped_forecast['yhat_upper']-grouped_forecast['y'])/grouped_forecast['y'])*100

        # KPI aggregate
        tot_value_by_month = df.groupby(['year', 'month'])['y'].sum().to_frame().reset_index()
        tot_prev_season = df['y'].loc[(df['ds']>forecast_from_tya0)&(df['ds']<=forecast_to_tya)].sum()
        delta = int(grouped_forecast['y'].iloc[-n_months:].sum()) - tot_prev_season
        delta_share = (delta/tot_prev_season)*100

        # merging
        final_forecast = grouped_forecast.append(tot_value_by_month)
        pivot = final_forecast.pivot_table(values = 'y', index = 'month', columns = 'year', fill_value=0, margins=False )
        pivot = pivot.astype(int)
        pivot = pivot.applymap('{:,.0f}'.format)

        #print
        print('-----------------------------------------------------------------------------------------')
        print('-----------------------------------------------------------------------------------------')
        print('The total predicted value of the UPCOMING season is {:,}'.format(int(grouped_forecast['y'].iloc[-n_months:].sum())))
        print('The total value of the PREVIOUS CLOSED season is {:,}'.format(int(tot_prev_season)))
        print('The differennce between the FORECASTED season and the PREVIOUS one is {:,}'.format(int(delta)))
        print('Expressed in % is {:,}'.format(round(delta_share, 2)))
        if regressors != None and len(regressors)>1:
            print('The parameters multiplied to the regressors are:{}'.format(regressors))
            print(future.loc[future['ds']>=forecast_from_tya])
        elif regressors != None and len(regressors)==1:
            print('The parameter multiplied to the regressors is:{}'.format(regressors))
            print(future.loc[future['ds']>=forecast_from_tya])
        print()
        print(pivot)
        print()
        print(grouped_forecast.iloc[-n_months:])
        #print(df.tail(10))
        
        table = go.Figure(data=[go.Table(
            header=dict(values=list(pivot.columns),
                    fill_color='paleturquoise',
                    align='left'),
            cells=dict(values=[pivot[c] for c in pivot.columns],
                fill_color='lavender',
                align='left'))
        ])
        # plot table 
        table.update_layout(width=1000, height=500)
        table.show()

        # plot graph 
        fig = plot_plotly(m, forecast)
        fig.show();

        # plot bars
        grouped_month_fig_forecast = px.bar(grouped_forecast.loc[grouped_forecast['ds']>=forecast_from], barmode='group', x='ds', y='y', color = 'month', title=title)
        grouped_month_fig_forecast.add_trace(go.Bar(
                  x = grouped_df['ds'].loc[(grouped_df['ds']>=forecast_from_tya0)&(grouped_df['ds']<=forecast_to_tya)],
                  y = grouped_df['real'].loc[(grouped_df['ds']>=forecast_from_tya0)&(grouped_df['ds']<=forecast_to_tya)],
                  name = 'fatturato stagione precedente'
                 ))
        grouped_month_fig_forecast.update_layout(legend_y=1.1, xaxis_type = 'category')
        grouped_month_fig_forecast.show();
        # plot compare

        # plot compoents
        if regressors != None:
            print(regressor_coefficients(m))
        m.plot_components(forecast)
           
    # forecast_to oltre storico dati stagione precente (no paragone) e forecst_from da entro un mese da ultima data nel df
    if (forecast_to - relativedelta(years=1)) > df['ds'].iloc[-1] and forecast_from <= df['ds'].iloc[-1] + relativedelta(days=1):
        # MULTIVARIATE
        # what if
        if regressors != None:
            for reg in regressors:
                m.add_regressor(reg,
                                mode=mode
                                )

        # forecasted 
        if to_forecast != None:
            def predictRegressor(df, x):
                df = df[["ds", x]]
                new_names = {x: "y"}
                df.rename(columns=new_names, inplace=True)

                m = Prophet()
                m.fit(df)

                #Create dates df for the future
                if freq=='D':
                    future = m.make_future_dataframe(periods = future_days, freq=freq)
                elif freq=='M':
                    future = m.make_future_dataframe(periods = n_months, freq=freq)                

                #Model predictions
                forecast = m.predict(future)

                # plot
                print('The forecasted regressor:')
                fig = plot_plotly(m, forecast)
                fig.show();
                print(('-----------------------------------------------------------------------------------------'))
                forecast.rename({'yhat':x}, axis=1, inplace=True)
                return forecast 

            for reg2 in to_forecast:
                m.add_regressor(reg2)

        # FITTING
        m.fit(df)

        # creating future df
        if freq=='D':
            future = m.make_future_dataframe(periods = future_days, freq=freq)
        elif freq=='M':
            future = m.make_future_dataframe(periods = n_months, freq=freq)

        future['floor']=floor
        future['cap']=cap

        # add additional regressors column
        if regressors != None:
            for reg, value in regressors.items():
                future = future.merge(df[[reg, 'ds']], on='ds',  how='left')
                if freq=='M':
                    future[reg].iloc[-n_months:] = df[reg].loc[(df['ds']>=forecast_from_tya)&(df['ds']<=forecast_to_tya)]*value
                elif freq=='D':
                    future[reg].iloc[-future_days:] = df[reg].loc[(df['ds']>=forecast_from_tya)&(df['ds']<=forecast_to_tya)]*value

        if replace_regressors_values != None:
            rrv(future, replace_regressors_values)

        if to_forecast != None:
            for reg3 in to_forecast:
                forecasted = predictRegressor(df, reg3)
                future[reg3] = forecasted[reg3]
        # predicting
        forecast = m.predict(future)

        # creating columns for groupby
        df['month'] = df['ds'].dt.month
        df['year'] = df['ds'].dt.year
        forecast['month'] = forecast['ds'].dt.month
        forecast['year'] = forecast['ds'].dt.year

        # creating df for results
        grouped_forecast = forecast\
            .loc[(forecast['ds']>=df['ds'].iloc[-1] + relativedelta(days=1)) & (forecast['ds']<=forecast_to)]\
            .groupby(['year', 'month'])['yhat', 'yhat_lower', 'yhat_upper', 'ds']\
            .agg({
                'yhat':'sum',
                'yhat_lower':'sum',
                'yhat_upper':'sum',
                'ds':'max'
                })\
                .reset_index()\
                .rename({'yhat':'y'}, axis=1)

        grouped_df = df\
            .groupby(['year', 'month'])['ds','y']\
            .agg({'ds':'max','y':'sum'}).reset_index().rename({'y':'real'}, axis=1)       

        grouped_forecast['lower_bound_%'] = ((grouped_forecast['yhat_lower']-grouped_forecast['y'])/grouped_forecast['y'])*100
        grouped_forecast['upper_bound_%'] = ((grouped_forecast['yhat_upper']-grouped_forecast['y'])/grouped_forecast['y'])*100

        # KPI aggregate
        tot_value_by_month = df.groupby(['year', 'month'])['y'].sum().to_frame().reset_index()
        tot_prev_season = df['y'].loc[(df['ds']>forecast_from_tya)&(df['ds']<=forecast_to_tya)].sum()
        #tot_prev_season=tot_prev_season.iloc[-n_months:].sum()
        delta = int(grouped_forecast['y'].iloc[-n_months:].sum()) - tot_prev_season
        delta_share = (delta/tot_prev_season)*100

        # merging
        final_forecast = grouped_forecast.append(tot_value_by_month)
        pivot = final_forecast.pivot_table(values = 'y', index = 'month', columns = 'year', fill_value=0, margins=False )
        pivot = pivot.astype(int)
        pivot = pivot.applymap('{:,.0f}'.format)

        #print
        print('-----------------------------------------------------------------------------------------')
        print('-----------------------------------------------------------------------------------------')
        print('The total predicted value of the UPCOMING season is {:,}'.format(int(grouped_forecast['y'].iloc[-n_months:].sum())))
        print('The total value of the PREVIOUS CLOSED season is {:,}'.format(int(tot_prev_season)))
        print('The differennce between the FORECASTED season and the PREVIOUS one is {:,}'.format(int(delta)))
        print('Expressed in % is {:,}'.format(round(delta_share, 2)))
        if regressors != None and len(regressors)>1:
            print('The parameters multiplied to the regressors are:{}'.format(regressors))
            print(future.loc[future['ds']>=forecast_from_tya])
        elif regressors != None and len(regressors)==1:
            print('The parameter multiplied to the regressors is:{}'.format(regressors))
            print(future.loc[future['ds']>=forecast_from_tya])
        print()
        print(pivot)
        print()
        print(grouped_forecast.iloc[-n_months:])
        #print(df.tail(10))
        
        table = go.Figure(data=[go.Table(
            header=dict(values=list(pivot.columns),
                    fill_color='paleturquoise',
                    align='left'),
            cells=dict(values=[pivot[c] for c in pivot.columns],
                fill_color='lavender',
                align='left'))
        ])
        # plot table 
        table.update_layout(width=1000, height=500)
        table.show()

        # plot graph 
        fig = plot_plotly(m, forecast)
        fig.show();

        # plot bars
        grouped_month_fig_forecast = px.bar(grouped_forecast.loc[grouped_forecast['ds']>=forecast_from], barmode='group', x='ds', y='y', color = 'month', title=title)
        grouped_month_fig_forecast.add_trace(go.Bar(
                  x = grouped_df['ds'].loc[(grouped_df['ds']>=forecast_from_tya)&(grouped_df['ds']<=forecast_to_tya)],
                  y = grouped_df['real'].loc[(grouped_df['ds']>=forecast_from_tya)&(grouped_df['ds']<=forecast_to_tya)],
                  name = 'fatturato stagione precedente'
                 ))
        grouped_month_fig_forecast.update_layout(legend_y=1.1, xaxis_type = 'category')
        grouped_month_fig_forecast.show();
        # plot compare

        # plot compoents
        if regressors != None:
            print(regressor_coefficients(m))
        m.plot_components(forecast)
    
    # forecast_to entro storico dati stagione precente (si paragone) e forecst_from da oltre ultima data nel df
    if (forecast_to - relativedelta(years=1)) <= df['ds'].iloc[-1] and forecast_from > df['ds'].iloc[-1] + relativedelta(days=1):

        # MULTIVARIATE
        # what if
        if regressors != None:
            for reg in regressors:
                m.add_regressor(reg,
                                mode=mode
                                )

        # forecasted 
        if to_forecast != None:
            def predictRegressor(df, x):
                df = df[["ds", x]]
                new_names = {x: "y"}
                df.rename(columns=new_names, inplace=True)

                m = Prophet()
                m.fit(df)

                #Create dates df for the future
                if freq=='D':
                    future = m.make_future_dataframe(periods = future_days0, freq=freq)
                elif freq=='M':
                    future = m.make_future_dataframe(periods = n_months0, freq=freq)                

                #Model predictions
                forecast = m.predict(future)

                # plot
                print('The forecasted regressor:')
                fig = plot_plotly(m, forecast)
                fig.show();
                print(('-----------------------------------------------------------------------------------------'))
                forecast.rename({'yhat':x}, axis=1, inplace=True)
                return forecast 

            for reg2 in to_forecast:
                m.add_regressor(reg2)

        # FITTING
        m.fit(df)

        # creating future df
        if freq=='D':
            future = m.make_future_dataframe(periods = future_days0, freq=freq)
        elif freq=='M':
            future = m.make_future_dataframe(periods = n_months0, freq=freq)

        future['floor']=floor
        future['cap']=cap
        
        # add additional regressors column
        if regressors != None:
            for reg, value in regressors.items():
                future = future.merge(df[[reg, 'ds']], on='ds',  how='left')
                if freq=='M':
                    future[reg].iloc[-n_months0:] = df[reg].loc[(df['ds']>forecast_from_oya)&(df['ds']<=forecast_to_oya)]*value
                elif freq=='D':
                    future[reg].iloc[-future_days0:] = df[reg].loc[(df['ds']>forecast_from_oya)&(df['ds']<=forecast_to_oya)]*value

        if replace_regressors_values != None:
            rrv(future, replace_regressors_values)

        if to_forecast != None:
            for reg3 in to_forecast:
                forecasted = predictRegressor(df, reg3)
                future[reg3] = forecasted[reg3]
        # predicting
        forecast = m.predict(future)

        # creating columns for groupby
        df['month'] = df['ds'].dt.month
        df['year'] = df['ds'].dt.year
        forecast['month'] = forecast['ds'].dt.month
        forecast['year'] = forecast['ds'].dt.year

        # creating df for results
        grouped_forecast = forecast\
            .loc[(forecast['ds']>=df['ds'].iloc[-1] + relativedelta(days=1)) & (forecast['ds']<=forecast_to)]\
            .groupby(['year', 'month'])['yhat', 'yhat_lower', 'yhat_upper', 'ds']\
            .agg({
                'yhat':'sum',
                'yhat_lower':'sum',
                'yhat_upper':'sum',
                'ds':'max'
                })\
                .reset_index()\
                .rename({'yhat':'y'}, axis=1)

        grouped_forecast['lower_bound_%'] = ((grouped_forecast['yhat_lower']-grouped_forecast['y'])/grouped_forecast['y'])*100
        grouped_forecast['upper_bound_%'] = ((grouped_forecast['yhat_upper']-grouped_forecast['y'])/grouped_forecast['y'])*100

        # KPI aggregate
        tot_value_by_month = df.groupby(['year', 'month'])['y'].sum().to_frame().reset_index()
        if freq=='M':
            tot_prev_season = df['y'].loc[(df['ds']>=forecast_from_oya0)&(df['ds']<=forecast_to_oya)].sum()
        elif freq=='D':
            tot_prev_season = df['y'].loc[(df['ds']>=forecast_from_oya0)&(df['ds']<=forecast_to_oya)].sum()
            #tot_prev_season=tot_prev_season.iloc[-n_months:].sum()
        delta = int(grouped_forecast['y'].iloc[-n_months:].sum()) - tot_prev_season
        delta_share = (delta/tot_prev_season)*100

        # merging
        final_forecast = grouped_forecast.append(tot_value_by_month)
        pivot = final_forecast.pivot_table(values = 'y', index = 'month', columns = 'year', fill_value=0, margins=False )
        pivot = pivot.astype(int)
        pivot = pivot.applymap('{:,.0f}'.format)

        #print
        print('-----------------------------------------------------------------------------------------')
        print('-----------------------------------------------------------------------------------------')
        print('The total predicted value of the UPCOMING season is {:,}'.format(int(grouped_forecast['y'].iloc[-n_months:].sum())))
        print('The total value of the PREVIOUS season is {:,}'.format(int(tot_prev_season)))
        print('The differennce between the FORECASTED season and the PREVIOUS one is {:,}'.format(int(delta)))
        print('Expressed in % is {:,}'.format(round(delta_share, 2)))
        if regressors != None and len(regressors)>1:
            print('The parameters multiplied to the regressors are:{}'.format(regressors))
            print(future.loc[future['ds']>=forecast_from_oya])
        elif regressors != None and len(regressors)==1:
            print('The parameter multiplied to the regressors is:{}'.format(regressors))
            print(future.loc[future['ds']>=forecast_from_oya])
        print()
        print(pivot)
        print()
        print(grouped_forecast.iloc[-n_months:])
        #print(df.tail(10))
        
        table = go.Figure(data=[go.Table(
            header=dict(values=list(pivot.columns),
                    fill_color='paleturquoise',
                    align='left'),
            cells=dict(values=[pivot[c] for c in pivot.columns],
                fill_color='lavender',
                align='left'))
        ])
        # plot table 
        table.update_layout(width=1000, height=500)
        table.show()

        # plot graph 
        fig = plot_plotly(m, forecast)
        fig.show();

        # plot bars
        grouped_month_fig_forecast = px.bar(grouped_forecast.loc[grouped_forecast['ds']>=forecast_from], barmode='group', x='month', y='y', color = 'month', title=title)
        grouped_month_fig_forecast.add_trace(go.Bar(
                  x = df['month'].loc[(df['ds']>=forecast_from_oya0)&(df['ds']<=forecast_to_oya)],
                  y = df['y'].loc[(df['ds']>=forecast_from_oya0)&(df['ds']<=forecast_to_oya)],
                  name = 'fatturato stagione precedente'
                 ))
        grouped_month_fig_forecast.update_layout(legend_y=1.1, xaxis_type = 'category')
        grouped_month_fig_forecast.show();
        # plot compare

        # plot compoents
        if regressors != None:
            print(regressor_coefficients(m))
        m.plot_components(forecast)

            
    # forecast_to entro storico dati stagione precente (si paragone) e forecst_from da giorno dopo ultima data nel df
    if forecast_to - relativedelta(years=1) <= df['ds'].iloc[-1] and forecast_from == df['ds'].iloc[-1] + relativedelta(days=1):

        if regressors != None:
            for reg in regressors:
                m.add_regressor(reg,
                                mode=mode
                                )

        # forecasted 
        if to_forecast != None:
            def predictRegressor(df, x):
                df = df[["ds", x]]
                new_names = {x: "y"}
                df.rename(columns=new_names, inplace=True)

                m = Prophet()
                m.fit(df)

                #Create dates df for the future
                if freq=='D':
                    future = m.make_future_dataframe(periods = future_days, freq=freq)
                elif freq=='M':
                    future = m.make_future_dataframe(periods = n_months, freq=freq)                

                #Model predictions
                forecast = m.predict(future)

                # plot
                print('The forecasted regressor:')
                fig = plot_plotly(m, forecast)
                fig.show();
                print(('-----------------------------------------------------------------------------------------'))
                forecast.rename({'yhat':x}, axis=1, inplace=True)
                return forecast 

            for reg2 in to_forecast:
                m.add_regressor(reg2)

        # FITTING
        m.fit(df)

        # creating future df
        if freq=='D':
            future = m.make_future_dataframe(periods = future_days, freq=freq)
        elif freq=='M':
            future = m.make_future_dataframe(periods = n_months, freq=freq)

        future['floor']=floor
        future['cap']=cap


        # add additional regressors column
        if regressors != None:
            for reg, value in regressors.items():
                future = future.merge(df[[reg, 'ds']], on='ds',  how='left')
                if freq=='M':
                    future[reg].iloc[-n_months:] = df[reg].loc[(df['ds']>=forecast_from_oya)&(df['ds']<=forecast_to_oya)]*value
                elif freq=='D':
                    future[reg].iloc[-future_days:] = df[reg].loc[(df['ds']>=forecast_from_oya)&(df['ds']<=forecast_to_oya)]*value             
        
        if replace_regressors_values != None:
            rrv(future, replace_regressors_values)

        if to_forecast != None:
            for reg3 in to_forecast:
                forecasted = predictRegressor(df, reg3)
                future[reg3] = forecasted[reg3]
        # predicting
        forecast = m.predict(future)

        # creating columns for groupby
        df['month'] = df['ds'].dt.month
        df['year'] = df['ds'].dt.year
        forecast['month'] = forecast['ds'].dt.month
        forecast['year'] = forecast['ds'].dt.year

        # creating df for results
        grouped_forecast = forecast\
            .loc[(forecast['ds']>=df['ds'].iloc[-1] + relativedelta(days=1)) & (forecast['ds']<=forecast_to)]\
            .groupby(['year', 'month'])['yhat', 'yhat_lower', 'yhat_upper', 'ds']\
            .agg({
                'yhat':'sum',
                'yhat_lower':'sum',
                'yhat_upper':'sum',
                'ds':'max'
                })\
                .reset_index()\
                .rename({'yhat':'y'}, axis=1)

        grouped_forecast['lower_bound_%'] = ((grouped_forecast['yhat_lower']-grouped_forecast['y'])/grouped_forecast['y'])*100
        grouped_forecast['upper_bound_%'] = ((grouped_forecast['yhat_upper']-grouped_forecast['y'])/grouped_forecast['y'])*100

        # KPI aggregate
        tot_value_by_month = df.groupby(['year', 'month'])['y'].sum().to_frame().reset_index()
        tot_prev_season = df['y'].loc[(df['ds']>=forecast_from_oya)&(df['ds']<=forecast_to_oya)].sum()
        #tot_prev_season=tot_prev_season.iloc[-n_months:].sum()
        delta = int(grouped_forecast['y'].iloc[-n_months:].sum()) - tot_prev_season
        delta_share = (delta/tot_prev_season)*100

        # merging
        final_forecast = grouped_forecast.append(tot_value_by_month)
        pivot = final_forecast.pivot_table(values = 'y', index = 'month', columns = 'year', fill_value=0, margins=False )
        pivot = pivot.astype(int)
        pivot = pivot.applymap('{:,.0f}'.format)

        #print
        print('-----------------------------------------------------------------------------------------')
        print('-----------------------------------------------------------------------------------------')
        print('The total predicted value of the UPCOMING season is {:,}'.format(int(grouped_forecast['y'].iloc[-n_months:].sum())))
        print('The total value of the PREVIOUS season is {:,}'.format(int(tot_prev_season)))
        print('The differennce between the FORECASTED season and the PREVIOUS one is {:,}'.format(int(delta)))
        print('Expressed in % is {:,}'.format(round(delta_share, 2)))
        if regressors != None and len(regressors)>1:
            print('The parameters multiplied to the regressors are:{}'.format(regressors))
            print(future.loc[future['ds']>=forecast_from_oya])
        elif regressors != None and len(regressors)==1:
            print('The parameter multiplied to the regressors is:{}'.format(regressors))
            print(future.loc[future['ds']>=forecast_from_oya])
        print()
        print(pivot)
        print()
        print(grouped_forecast.iloc[-n_months:])
        #print(df.tail(10))
        
        table = go.Figure(data=[go.Table(
            header=dict(values=list(pivot.columns),
                    fill_color='paleturquoise',
                    align='left'),
            cells=dict(values=[pivot[c] for c in pivot.columns],
                fill_color='lavender',
                align='left'))
        ])
        # plot table 
        table.update_layout(width=1000, height=500)
        table.show()

        # plot graph 
        fig = plot_plotly(m, forecast)
        fig.show();

        # plot bars
        grouped_month_fig_forecast = px.bar(grouped_forecast.loc[grouped_forecast['ds']>=forecast_from], barmode='group', x='month', y='y', color = 'month', title=title)
        grouped_month_fig_forecast.add_trace(go.Bar(
                  x = df['month'].loc[(df['ds']>=forecast_from_oya)&(df['ds']<=forecast_to_oya)],
                  y = df['y'].loc[(df['ds']>=forecast_from_oya)&(df['ds']<=forecast_to_oya)],
                  name = 'fatturato stagione precedente'
                 ))
        grouped_month_fig_forecast.update_layout(legend_y=1.1, xaxis_type = 'category')
        grouped_month_fig_forecast.show();
        # plot compare

        # plot compoents
        if regressors != None:
            print(regressor_coefficients(m))
        m.plot_components(forecast)
    # --------------------------------------------------                       ©                       --------------------------------------------------
    # --------------------------------------------------                      ©©©                      --------------------------------------------------
    # --------------------------------------------------                     ©©©©©                     --------------------------------------------------
    # --------------------------------------------------                    ©©©©©©©                    --------------------------------------------------
    # --------------------------------------------------                   ©©©©©©©©©                   --------------------------------------------------
    # --------------------------------------------------                  ©©©©©©©©©©©                  --------------------------------------------------
    # --------------------------------------------------                 ©©©©©©©©©©©©©                 --------------------------------------------------
    # --------------------------------------------------                ©©©©©©©©©©©©©©©                --------------------------------------------------
    # --------------------------------------------------               ©©©©©©©©©©©©©©©©©               --------------------------------------------------
    # --------------------------------------------------              ©©©©©©©©©©©©©©©©©©©              --------------------------------------------------
    # --------------------------------------------------             ©©©©©©©©©©©©©©©©©©©©©             --------------------------------------------------
    # --------------------------------------------------            ©©©©©©©©©©©©©©©©©©©©©©©            --------------------------------------------------
    # --------------------------------------------------           ©©©©©©©©©©©©©©©©©©©©©©©©©           --------------------------------------------------
    # --------------------------------------------------          ©©©©©©©©©©©©©©©©©©©©©©©©©©©          --------------------------------------------------
    # --------------------------------------------------         ©©©©©©©©©©©©©©©©©©©©©©©©©©©©©         --------------------------------------------------
    # --------------------------------------------------        ©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©        --------------------------------------------------
    # --------------------------------------------------       ©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©       --------------------------------------------------
    # --------------------------------------------------      ©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©      --------------------------------------------------
    # --------------------------------------------------     ©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©     --------------------------------------------------
    # --------------------------------------------------    ©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©    --------------------------------------------------
    # --------------------------------------------------   ©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©   --------------------------------------------------
    # --------------------------------------------------  ©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©  --------------------------------------------------    
    # -------------------------------------------------- ©©©©©©©©©©©©©©©©© TEST CASE ©©©©©©©©©©©©©©©©© --------------------------------------------------
    if forecast_to <= df['ds'].iloc[-1]:

        df_train = df.loc[df['ds']<forecast_from]
        df_test = df.loc[(df['ds']>=forecast_from) & (df['ds']<=forecast_to)]

        # MULTIVARIATE
        # what if
        if regressors != None:
            for reg in regressors:
                m.add_regressor(reg)

        # forecasted 
        if to_forecast != None:
            def predictRegressor(df, x):
                df = df_train[["ds", x]]
                new_names = {x: "y"}
                df.rename(columns=new_names, inplace=True)

                m = Prophet()
                m.fit(df_train)

                #Create dates df for the future
                if forecast_from > (df['ds'].iloc[-1] + relativedelta(days=1)):
                    if freq=='D':
                        future = m.make_future_dataframe(periods = future_days0, freq=freq)
                    elif freq=='M':
                        future = m.make_future_dataframe(periods = n_months0, freq=freq)
                else:
                    if freq=='D':
                        future = m.make_future_dataframe(periods = future_days, freq=freq)
                    elif freq=='M':
                        future = m.make_future_dataframe(periods = n_months, freq=freq)

                #Model predictions
                forecast = m.predict(future)
                forecast.rename({'yhat':x}, axis=1, inplace=True)
                return forecast
                
            def univariate_graph(y_true, y_pred):
                fig = px.line(y_true, x='ds', y="y")
                fig.add_trace(go.Line(mode="lines", x=y_pred['ds'], y=y_pred['yhat'], name="Prediction"))
                fig.update_layout(legend_y=1.1, xaxis_type = 'category')
                fig.show()

            for reg2 in to_forecast:
                m.add_regressor(reg2)
        
        m.fit(df_train)
        
        # creating future df
        if freq=='D':
            future = m.make_future_dataframe(periods = future_days, freq=freq)
        elif freq=='M':
            future = m.make_future_dataframe(periods = n_months, freq=freq)

        future['floor']=floor
        future['cap']=cap

        # add additional regressors column
        if regressors != None:
            for reg, value in regressors.items():
                future = future.merge(df_train[[reg, 'ds']], on='ds',  how='left')
                if freq=='M':
                    future[reg].iloc[-n_months:] = df_test[reg]*value
                if freq=='D':
                    future[reg].iloc[-future_days:] = df_test[reg]*value

        if to_forecast != None:
            for reg3 in to_forecast:
                forecasted = predictRegressor(df, reg3)
                future[reg3] = forecasted[reg3]
                future[reg3].loc[df['ds']<forecast_from] = df_train[reg3]

        

        # change values
        if replace_regressors_values != None:
            rrv(future, replace_regressors_values)

        #print(future)
        # predicting
        forecast = m.predict(future)

        # defining KPIs
        pred_total_value = forecast['yhat'].loc[(forecast['ds']>=forecast_from) & (forecast['ds']<=forecast_to)].sum()
        real_total_value = df['y'].loc[(df['ds']>=forecast_from) & (df['ds']<=forecast_to)].sum()
        delta = pred_total_value - real_total_value

        # creating columns for groupby
        df['month'] = df['ds'].dt.month
        df['year'] = df['ds'].dt.year
        forecast['month'] = forecast['ds'].dt.month
        forecast['year'] = forecast['ds'].dt.year

        # creating single dfs
        tot_value_by_month = df.loc[(df['ds']>=forecast_from) & (df['ds']<=forecast_to)].groupby(['year', 'month'])['y'].sum().to_frame().reset_index()
        grouped_forecast = forecast.loc[(forecast['ds']>=forecast_from) & (forecast['ds']<=forecast_to)].groupby(['year', 'month'])['yhat', 'yhat_lower', 'yhat_upper', 'ds'].agg({'yhat':'sum','yhat_lower':'sum','yhat_upper':'sum','ds':'max'}).reset_index()
        grouped_forecast['lower_bound_%'] = ((grouped_forecast['yhat_lower']-grouped_forecast['yhat'])/grouped_forecast['yhat'])*100
        grouped_forecast['upper_bound_%'] = ((grouped_forecast['yhat_upper']-grouped_forecast['yhat'])/grouped_forecast['yhat'])*100

        # creating final df
        final_df = tot_value_by_month.merge(grouped_forecast, on = ['year', 'month'], how = 'inner')
        final_df['dlt(pred-real)'] = final_df['yhat'] - final_df['y']
        final_df['%'] = (final_df['dlt(pred-real)']/final_df['y'])*100;
        final_df['wape %'] = final_df['dlt(pred-real)']/np.sum(final_df['y'])*100
        
        # final_df2 è il df formattato a stringa per visualizzare meglio i numeri
        final_df2 = final_df.astype(int)
        final_df2 = final_df.applymap('{:,.0f}'.format)
        
        ma_pe = mape(final_df['y'], final_df['yhat'])
        wa_pe = wape(final_df['y'], final_df['yhat'])
        wa_pe_mean = wape_mean(final_df['y'], final_df['yhat'])
    
        # printouts
        print('-----------------------------------------------------------------------------------------');
        print('-----------------------------------------------------------------------------------------');
        print('The total predicted value of the season is {:,} €'.format(int(pred_total_value)));
        print('The total real value of the season is {:,} €'.format(int(real_total_value)));
        print('The difference between the real value and the predicted one is {:,} €'.format(int(delta)));
        print('Expressed in % is: {:,}  '.format(int((delta/real_total_value)*100)))
        #print('The Mean Squared Error is {:,} %'.format(mean_squared_error(final_df['y'], final_df['yhat'])))
        print('The Root Mean Squared Error is {:,} %'.format(np.sqrt(mean_squared_error(final_df['y'], final_df['yhat']))))
        #print('The Mean Squared Log Error is {:,} %'.format(mean_squared_log_error(final_df['y'], final_df['yhat'])))
        print('The Mean Absolute Error is {:,} %'.format(mean_absolute_error(final_df['y'], final_df['yhat'])))
        print('The OVERALL MAPE is {}  %'.format((ma_pe)));
        print('The OVERALL WAPE  is {}  %'.format((wa_pe)));
        print('The OVERALL WAPE MEAN is {}  %'.format((wa_pe_mean)));
        if regressors != None and len(regressors)>1:
            print('The parameters multiplied to the regressors are:{}'.format(regressors))
            print(future.loc[future['ds']>=forecast_from])
        elif regressors != None and len(regressors)==1:
            print('The parameter multiplied to the regressors is:{}'.format(regressors))
            print(future.loc[future['ds']>=forecast_from])        
        print('-----------------------------------------------------------------------------------------');
        print('-----------------------------------------------------------------------------------------');
        print('The REAL monthly bdg is:');
        print(tot_value_by_month[-n_months:])
        print('-----------------------------------------------------------------------------------------');
        print('The PREDICTED monthly bdg is:');
        print(grouped_forecast[-n_months:])
        print('-----------------------------------------------------------------------------------------');
        print('FINAL DF')
        print(final_df[-n_months:]);
        print('-----------------------------------------------------------------------------------------');
        print('-----------------------------------------------------------------------------------------')
        
        table = go.Figure(data=[go.Table(
        header=dict(values=list(final_df2[['year','month','y','yhat','dlt(pred-real)','%','wape %']].columns),
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[final_df2.year, final_df2.month, final_df2['y'], final_df2.yhat, final_df2['dlt(pred-real)'], final_df2['%'], final_df2['wape %']],
                fill_color='lavender',
                align='left'))
        ])
        table.update_layout(width=1000, height=500)
        table.show()

        # plotting overall forecast
        fig = plot_plotly(m, forecast)
        fig.add_trace(go.Line(mode="lines", x=df['ds'].loc[(df['ds']>=forecast_from) & (df['ds']<=forecast_to)], y=df['y'].loc[(df['ds']>=forecast_from) & (df['ds']<=forecast_to)], name="Actual price"))
        fig.show();

        # plotting overall
        overall = px.bar(final_df[-n_months:], x='month', y=['yhat'], color = 'month', title=title, 
        barmode='group'
        )
        overall.add_trace(go.Bar(x = final_df['month'][-n_months:], y = final_df['y'][-n_months:], name = 'real', 
        ))
        overall.update_layout(legend_y=1.1, xaxis_type = 'category')
        overall.show()

        if regressors != None:
            m.plot_components(forecast)
        # plotting components
        m.plot_components(forecast)

        if to_forecast != None:
            if freq=='M':
                univariate_graph(final_df[['ds','y']][-n_months:], final_df[['ds','yhat']][-n_months:])
            elif freq=='D':
                univariate_graph(final_df[['ds','y']][-future_days:], final_df[['ds','yhat']][-future_days:])
